In [ ]:
import cv2
import deepface

from deepface.extendedmodels import Race

In [ ]:
face_min_height_scale = 5

In [ ]:
race_labels = ['asian', 'indian', 'black', 'white', 'middle eastern', 'latino hispanic']

In [ ]:
model = Race.loadModel()

In [ ]:
cap = cv2.VideoCapture('video1.mp4')

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
fps = round(fps)*10

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

face_min_height = int(height//face_min_height_scale)

In [ ]:
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
out = cv2.VideoWriter('video1_out.avi', fourcc, 1, (width, height))

In [ ]:
def ethnicity_detect(img, target_size=(224, 224)):
    img = cv2.resize(img, target_size)
    img_pixels = image.img_to_array(img)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255
    
    preds = model.predict(img_pixels)
    
    return race_labels[np.argmax(preds)]
    

In [ ]:
def face_detect(img):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,
                                          scaleFactor = 1.1,
                                          minNeighbors = 5,
                                          minSize = (face_min_height, face_min_height)
                                         )
    return faces
    

In [ ]:
def show_faces(img, faces):
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        
        face = img[y:y+h, x:x+w]
        
        race_label = ethnicity_detect(face)
        img = cv2.putText(img, race_label, (int(x),int(y+h)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
    return img

def cut_faces(img, faces):
    imgs = []
    for (x,y,w,h) in faces:
        imgs.append(img[y:y+h, x:x+w])
    return imgs

In [ ]:
count = 0
frame_face_count = 0

while(cap.isOpened()):
    cap.set(1, count)
    ret, frame = cap.read()
    
    if ret:
        
        faces = face_detect(frame)

        frame = show_faces(frame, faces)

        out.write(frame)

        count+=fps

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    else:
        break
        
cap.release()
out.release()